In [1]:
import mat_properties as prop


import os
import time
import cotel
import GTU
import mat_properties as prop
import numpy as n
import pandas as pd
from scipy.optimize import root


# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
fractiongas = (
    0.715784461314205,
    0.0114726415414296,
    0.143332198252282,
    0.116991486761544,
    0.0124192121305392
)


water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)


Methane = prop.Materials_prop(
    "METHANE",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)


fractionwaterMethane = (0.848859488304273, 0.151140511695727, 0, 0, 0)
waterMethanemix = "Water*METHANE"

waterMethane = prop.Materials_prop(
    waterMethanemix,
    fractionwaterMethane,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)

gas = prop.Materials_prop(
    gasmix,
    fractiongas,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)



gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)




from PKM import steam_transformer
water_streams.loc['DROSVD-ST','T':'G'] = [509.0071373,7.845,3423.525641,47.55878321]

ST = steam_transformer(stream11 = 'DROSVD-ST', 
                       water = water, 
                       water_streams = water_streams, 
                       heaters = heaters, 
                       Pdr1 = 2, 
                       Pdr2 = 0.8, 
                       P2 = 2, 
                       dT = 15, 
                       dTmin = 5, 
                       Tdec = 10)
steam_trans = ST.calc()


water_streams.loc['ST-GPK','T':'G'] = [steam_trans['T16'],steam_trans['P16'],steam_trans['H16'],steam_trans['G1']]
water_streams.loc['ST-PKM','T':'G'] = [steam_trans['T24'],steam_trans['P2'],steam_trans['H24'],steam_trans['G2']]
heaters.at['Strans','Qw'] = steam_trans['Q']


from PKM import reformer



# def calculate_reformer(G2, Trefromer_out=700, Pressure_reform=2):

#     Gmethane = (0.151140511695727/0.848859488304273)*G2
#     Greformer = Gmethane + G2

#     # Или не Pr (2000), а 2500 как в аспене?
#     Tmethane = 121.53615158005 #выход с компрессора
#     Hmet = Methane.p_t(Pressure_reform, Tmethane)['h']
#     H1r = (G2*H22 + Gmet*Hmet) / Greformer
#     H2r = waterMethane.p_t(Pressure_reform, Trefromer_out)['h']
#     Qdt = Greformer*(H2r-H1r)/1000
#     Qr = (52166.505091208484/44.6397313913235)*Greformer/1000
#     Qref = Qdt+Qr
#     T1gas = 1969
#     T2gas = 800
#     H1gas = gas.p_t(0.1, T1gas)['h']
#     H2gas = gas.p_t(0.1, T2gas)['h']
#     Ggas = Qref/(H1gas-H2gas)
#     Gair = Ggas*48.5966429877363/51.2672005145991
#     Gch4 = Ggas*2.67055752686283/51.2672005145991
#     res_out={'Greformer':Greformer,'Ggas':Ggas,'Gair':Gair,'Gch4':Gch4,'Qref':Qref,'Qdt':Qdt,'Qr':Qr}
#     return res_out

# Reform_out=calculate_reformer(G2)
# print(Reform_out)
ref = reformer(stream11 = 'ST-PKM',
               water = water,
               gas = gas,
               Methane = Methane,
               waterMethane = waterMethane,
               water_streams = water_streams,
               heaters = heaters,
               Tref = 700,
               Pref = 2)
reform = ref.calc()


heaters.at['Reformer','Qw'] = reform['Q']


print(reform['Q'],reform['Ggas'])

86823.68752359554 54.64992435481982
